In [95]:
from random import random, setstate, getstate
from math import log
import numpy as NP
from statistics import mean
import time
from tqdm import tqdm

def p(x):
  z = -(1/x)* log(random())
  return z

def f():
  T = 8                 #Tiempo que estará disponible el sistema
  pe = 3                #Parámetro para el proceso de entrada
  ps1 = 2               #Parámetro para el proceso de salida del servidor 1
  ps2 = 0.5               #Parámetro para el proceso de salida del servidor 2
  inf = 1000000
  t = 0                 #Tiempo actual del sistema
  te = p(pe)            #Tiempo en el que ocurre una entrada           
  c = 0                 #Contador de clientes del sistema           
  ts1 = inf
  ts2 = inf             #Tiempo en el que ocurre una salida
  SA = []               #Lista de los clientes que estan a la espera de ser atendidos
  TE1 = []              #Tiempos de entrada del servidor 1
  TE2 = []              #Tiempos de entrada del servidor 2
  TS1 = []              #Tiempos de salida del servidor 1
  TS2 = []              #Tiempos de salida del servidor 2
  TA1 = []              #Tiempos de atención en el servidor 1
  TA2 = []              #Tiempos de atención en el servidor 2

  while t<T or c>0:

    t = min(te, ts1, ts2)         #Variable que indica qué suceso sucedió primero, ya sea una entrada o salida

    if t == te:                   #Cosas que suceden si primero sucedió una llegada
      c = c + 1                   #Se aumenta en 1 el contador de clientes en el sistema
      if t>T:                     #Si ya se excedió el tiempo de simulación, no se permiten más entradas
        te = inf
      else:
        te = te + p(pe)

      if c == 1:                  #Si sólo hay un cliente, éste puede elegir entre los dos servidores disponibles
        if random()<0.5:          #Entró en el servidor 1
          ts1 = t + p(ps1)        #Se actualiza el tiempo de salida del servidor 1
          TE1.append(t)           #Se agrega el tiempo en que entró al servidor
          TA1.append(t)           #Como sólo hay un cliente, se le atiende de forma inmediata
        else:
          ts2 = t + p(ps2)        #Se actualiza el tiempo de salida del servidor 2
          TE2.append(t)           #Se agrega el tiempo en que entró al servidor
          TA2.append(t)           #Como sólo hay un cliente, se le atiende de forma inmediata
      
      if c == 2:                  #Si ya hay un cliente, éste entrará en el servidor disponible
        if(ts1<inf):
          ts2 = t + p(ps2)
          TE2.append(t)
          TA2.append(t)
        else:
          ts1 = t + p(ps1)
          TE1.append(t)  
          TA1.append(t)   
      #print("%.4f"%t ,c)

      if c>2:                   #Si ya hay mas de dos cliente, se agrega el siguiente tiempo en una lista de espera
        SA.append(t)

    elif t == ts1:              #Lo siguiente ocurre cuando sucedió una salida del servidor 1
      ts1 = t + p(ps1)          #Se actualiza el siguiente tiempo de salida
      c = c - 1                 #Se actualiza el número de clientes en el sistema
      TS1.append(t)             #Se guarda el tiempo en que sucedió la salida
      #print("             %.4f"%t,c)

      if c>=2:                  #Si hay mas de dos personas en el servidor, cuando sale alguien, el siguiente en línea es atendido inmediatamente
        TA1.append(t)

      if c>1:
        ts1 = t + p(ps1)        #Si hay más clientes se actualiza el siguiente tiempo de espera
      else:
        ts1 = inf               #Si ya no hay cliente, se marca un tiempo de slaida infinito

      if c>=2:
        TE1.append(SA[0])       #Como ya se liberó este servidor se guarda el tiempo en que entró el siguiente cliente
        SA.pop(0)               #Se elimina al cliente de la lista de espera

    elif t == ts2:
      ts2 = t + p(ps2)
      c = c - 1
      TS2.append(t)
      #print("                          %.4f"%t,c)

      if c>=2:
        TA2.append(t)
      if c>1:
        ts2 = t + p(ps2)
      else:
        ts2 = inf
      
      if c>=2:
        TE2.append(SA[0])
        SA.pop(0)

  s1 = NP.subtract(TS1,TE1)         #Vector de la resta de los tiempos correspondientes de salida y entrada del servidor 1
  s2 = NP.subtract(TS2,TE2)         #Vector de la resta de los tiempos correspondientes de salida y entrada del servidor 2 
  a1 = NP.subtract(TS1,TA1)         #Vector de la resta de los tiempos correspondientes de salida y atención del servidor 1
  a2 = NP.subtract(TS2,TA2)         #Vector de la resta de los tiempos correspondientes de salida y atención del servidor 2
  return [mean(s1), mean(s2), mean(a1), mean(a2)]

nn = 100000  
S1 = []
S2 = []
A1 = []
A2 = []
for i in tqdm(range(nn)):  #Ciclo que guarda todos los tiempos promedio de las nn simulaciones
  G = f()
  S1.append(G[0])
  S2.append(G[1])
  A1.append(G[2])
  A2.append(G[3])

print("\n\n")
print("El tiempo medio de estancia en el servidor 1 es:  %1.4f" %mean(S1), " horas")
print("El tiempo medio de estancia en el servidor 2 es:  %1.4f" %mean(S2), " horas")
print("El timepo promedio de atención del servidor 1 es: %1.4F" %mean(A1), " horas")
print("El timepo promedio de atención del servidor 2 es: %1.4F" %mean(A2), " horas")
#print(TE1, TS1)


100%|██████████| 100000/100000 [00:40<00:00, 2467.08it/s]





El tiempo medio de estancia en el servidor 1 es:  1.9356
El tiempo medio de estancia en el servidor 2 es:  3.4862
El timepo promedio de atención del servidor 1 es: 0.5047
El timepo promedio de atención del servidor 2 es: 2.3083
